In [8]:
import numpy as np
import pandas as pd

In [9]:
def run_markov_to_convergence(transitions, start_state=None, tol=1e-6, max_steps=1000):
    """
    Проганяє ланцюг Маркова до збіжності зі словника переходів.
    
    transitions: dict
        словник переходів, наприклад:
        {
            "Landing": {"Signup": 0.6, "Exit": 0.4},
            "Signup": {"Onboarding": 0.7, "Exit": 0.3},
            ...
        }
    start_state: str або None
        Початковий стан. Якщо None, бере перший ключ словника.
    tol: float
        Поріг збіжності.
    max_steps: int
        Максимальна кількість кроків.
    
    Повертає:
        final_dist: pd.Series, довгостроковий розподіл по станах
        states: list, список станів
        P: numpy.ndarray, матриця переходів
    """
    # --- Побудова матриці та індексів ---
    states = list(transitions.keys())
    state_idx = {s: i for i, s in enumerate(states)}
    
    P = np.zeros((len(states), len(states)))
    for from_state, to_states in transitions.items():
        i = state_idx[from_state]
        for to_state, prob in to_states.items():
            j = state_idx[to_state]
            P[i, j] = prob
    
    # Перевірка сум рядків
    for i, row in enumerate(P):
        if not np.isclose(row.sum(), 1.0):
            raise ValueError(f"Сума ймовірностей для стану {states[i]} не дорівнює 1")
    
    # --- Початковий розподіл ---
    if start_state is None:
        start_state = states[0]
    pi = np.zeros(len(states))
    pi[state_idx[start_state]] = 1.0
    
    # --- Проганяємо до збіжності ---
    for step in range(max_steps):
        pi_new = pi @ P
        if np.max(np.abs(pi_new - pi)) < tol:
            print(f"Збіжність досягнута на кроці {step+1}")
            break
        pi = pi_new
    
    final_dist = pd.Series(pi_new, index=states)
    return final_dist, states, P

In [10]:
transitions = {
    "Landing":      {"Signup": 0.6, "Exit": 0.4},
    "Signup":       {"Onboarding": 0.7, "Exit": 0.3},
    "Onboarding":   {"Signup": 0.1, "Add_to_cart": 0.6, "Exit": 0.3},
    "Add_to_cart":  {"Onboarding": 0.2, "Checkout": 0.4, "Exit": 0.4},
    "Checkout":     {"Success": 0.7, "Exit": 0.3},
    "Success":      {"Success": 1.0},
    "Exit":         {"Exit": 1.0},
}

final_dist, states, P = run_markov_to_convergence(transitions)
print(final_dist.round(3))

Збіжність досягнута на кроці 19
Landing        0.000
Signup         0.000
Onboarding     0.000
Add_to_cart    0.000
Checkout       0.000
Success        0.087
Exit           0.913
dtype: float64


In [11]:
states

['Landing',
 'Signup',
 'Onboarding',
 'Add_to_cart',
 'Checkout',
 'Success',
 'Exit']

In [12]:
P

array([[0. , 0.6, 0. , 0. , 0. , 0. , 0.4],
       [0. , 0. , 0.7, 0. , 0. , 0. , 0.3],
       [0. , 0.1, 0. , 0.6, 0. , 0. , 0.3],
       [0. , 0. , 0.2, 0. , 0.4, 0. , 0.4],
       [0. , 0. , 0. , 0. , 0. , 0.7, 0.3],
       [0. , 0. , 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 1. ]])